In [1]:
import random
import numpy as np
from scipy import special
from scipy.stats import binom
import scipy
import time
import multiprocessing as mp
import itertools
from putty_mod_FNs_001 import rec_balls, sig_cor, putty_draws, runs_nash_counter, avg_succ
np.set_printoptions(suppress=True)












In [3]:



def putty_tile(n):


    #for putty rewrite

    #need to make sure a different seed is used each time we run the function
    random.seed()
    np.random.seed()
    scipy.random.seed()



    # number of signalers
    numsig = 2
    # number of urns for each signaler
    numsigurns = 6
    # number of terms for each signaler
    numsigterms = 2

    # number of signalers
    exenumsig = 1
    # number of urns for each signaler
    exenumsigurns = 6
    # number of terms for each signaler
    exenumsigterms = 2


    # runs
    runs = 200
    # number of basic actions is the numrecterms
    numrecterms = 4
    exenumrecterms = 2
    # plays per run
#     plays = 3*10**7
    plays = 10**7
    # number of times that learning dynamic alternates
    iterations = 10
    alternations = iterations*2
    alt_list = list(range(0, plays+(plays // alternations), plays // alternations))
    # print('alt_list')
    # print(alt_list)
    # amount of reinforcement and punishment on each alternation
#     reinforcement = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
#     punishment = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
#     reinforcement = np.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
#     punishment = np.array([0, -4, 0, -4, 0, -4, 0, -4, 0, -4, 0]) 
    
    reinarray = np.array([3, 3])
    punarray = np.array([-3, -3])
    reinforcement = np.tile(reinarray, alternations+1)
    punishment = np.tile(punarray, alternations+1)
    
    
    #sig cost for only part of the punishment comming from costly signals
#     sigcostarray = np.array([-0.5, -0.5, -0.5, -0.5])
    sigcostarray = np.array([0, 0, 0, 0])
    signalcost = np.tile(sigcostarray, alternations+1)
#     sig_cost = -0.5
    
    # bound for bounded reinforcement
    bound = 10**5
    # cutoff for measuring success
    cutoffA = 0.92
    cutoffB = 0.8
    cutoffC = 0.6



    # signaler weights

    sig_weights = np.ones([runs*numsig*numsigurns, numsigterms])

    exesig_weights = np.ones([runs*exenumsig*exenumsigurns, exenumsigterms])

    # print('sig_weights')
    # print(sig_weights)


    # number of receiver terms is going to be the same as the total number of statements that can be transmitted
    numrecurns = 4
    exenumrecurns = 2


    recweights = np.ones([runs, numrecurns, numrecterms])
    exerecweights =np.ones([runs, exenumrecurns, exenumrecterms])

    # print('recweights')
    # print(recweights)

    #create an array to iterate recurns index
    recit = np.arange(runs)
    recit = np.reshape(recit, [runs,1])
    #similar array for iterating signalers
    sigit = np.arange(numsig)

    sigit = np.tile(sigit, runs)
    # print('sigit')
    # print(sigit)

    sigit2 = np.arange(runs)
    sigit2 = np.repeat(sigit2, numsig)
    # print(sigit)
    # print(sigit2)
    # sigit = np.reshape(sigit, [numsig, 1])
    sigit = np.vstack((sigit2, sigit))
    # print(sigit)

    #create an array to iterate exerecurns index
    exerecit = np.arange(runs)
    exerecit = np.reshape(exerecit, [runs,1])
    #similar array for iterating EXEsignalers
    exesigit = np.arange(exenumsig)

    exesigit = np.tile(exesigit, runs)
    # print('sigit')
    # print(sigit)

    exesigit2 = np.arange(runs)
    exesigit2 = np.repeat(exesigit2, exenumsig)
    # print(sigit)
    # print(sigit2)
    # sigit = np.reshape(sigit, [numsig, 1])
    exesigit = np.vstack((exesigit2, exesigit))
    # print(sigit)





    # cumulative success
    cumsuc = np.zeros([runs, 1])
    cumsucA = np.zeros([runs, 1])
    cumsucB = np.zeros([runs, 1])
    cumsucC = np.zeros([runs, 1])
    
    # print('cumsuc')
    # print(cumsuc)



    pickslist = np.arange(runs*numsig)
    picksEXSlist = np.arange(runs)

    # print('pickslist')
    # print(pickslist)

    drawslist = np.arange(runs)
    exedrawslist = np.arange(runs)




    indexedrecurns = np.reshape(np.arange(numrecurns), [2, 2])
    # print('indexedrecurns')
    # print(indexedrecurns)
    # indexedexerecurns = np.reshape(np.arange(exenumrecurns), [2, 2])
    indexedexerecurns = np.arange(exenumrecurns)

    # print('indexedexerecurns')
    # print(indexedexerecurns)




    for it_00 in range(0, alternations):

        rein = reinforcement[it_00]
        punish = punishment[it_00]
        sig_cost = signalcost[it_00]

        for it_01 in range(alt_list[it_00], alt_list[it_00+1]):


            # determine a state of nature at random with equal probability
            nature = np.random.randint(0, numsigurns, runs)
    #         print('nature')
    #         print(nature)
    #         now the signalers must pick from the appropriate urns
            # lets get the weights for each of these urns

            sigit3 = np.vstack((sigit, np.repeat(nature, numsig)))
            exesigit3 = np.vstack((exesigit, np.repeat(nature, exenumsig)))
            #     print('sigit3')
            #     print(sigit3)

            #################################################################################
            # 
            ##################################################


            sigweight_id = np.ravel_multi_index(sigit3, (runs, numsig, numsigurns))
            #     print('sigweight_id')
            #     print(sigweight_id)
            sigweight4pick = sig_weights.copy()
            #     print('sigweight4pick')
            #     print(sigweight4pick)
            sigweight_play = sigweight4pick[sigweight_id]
            #     print('sigweight_play')
            #     print(sigweight_play)


            exesigweight_id = np.ravel_multi_index(exesigit3, (runs, exenumsig, exenumsigurns))
            # print('exesigweight_id')
            # print(exesigweight_id)
            exesigweight4pick = exesig_weights.copy()
            #     print('exesigweight4pick')
            #     print(exesigweight4pick)
            exesigweight_play = exesigweight4pick[exesigweight_id]
            #     print('exesigweight_play')
            #     print(exesigweight_play)




            # to do weighted picks in parallel, we're going to normalize to values in [0, 1)
            # then generate a random number and see where it falls in the cumulative sum
            # of the normalized weights. Then we can just do a boolean comparison to get 
            # discrete values
            sigweight_total = np.vstack(np.sum(sigweight_play, axis = 1))
            sigweight_norm = sigweight_play / sigweight_total
            sigweight_norm = np.cumsum(sigweight_norm, axis = 1)
            # print('sig weight norm')
            # print(sigweight_norm)
            rand4picks = np.random.rand(runs*numsig,1)
            # print('rand 4 picks')
            # print(rand4picks)
            picks = (rand4picks > sigweight_norm).astype(int)

            picks = np.sum(picks, axis=1)
            picks = np.reshape(picks, [runs, numsig])
    #         print('picks')
    #         print(picks)




            # to do weighted picksEXS in parallel, we're going to normalize to values in [0, 1)
            # then generate a random number and see where it falls in the cumulative sum
            # of the normalized weights. Then we can just do a boolean comparison to get 
            # discrete values
            exesigweight_total = np.vstack(np.sum(exesigweight_play, axis = 1))
            exesigweight_norm = exesigweight_play / exesigweight_total
            exesigweight_norm = np.cumsum(exesigweight_norm, axis = 1)
            # print('exesig weight norm')
            # print(exesigweight_norm)
            rand4picksEXS = np.random.rand(runs*exenumsig,1)
            # print('rand 4 picksEXS')
            # print(rand4picksEXS)
            picksEXS = (rand4picksEXS > exesigweight_norm).astype(int)

            picksEXS = np.sum(picksEXS, axis=1)
    #         print('picksEXS')
    #         print(picksEXS)
















            # now use picks to identify the appropriate receiver urn
            recurns = np.ravel_multi_index(picks.T, indexedrecurns.shape)
            recurns = np.vstack(recurns)


            recurns = np.reshape(recurns, [runs])
            # put in mod fn here
            #recurns needs change so that it is at random with equal probability when single term is transmitted
            #
            #
            # Here's a plan: when random single sender transmits 
            # random 10's of length runs pull which basic sender will send
            # set preliminar urns to 0 for 0 and 2 for 1
            # then add element wise a second random 01's list of length runs to cover the random choide of 0 or 1 
            # and 2 or 3 rec urns
            # lets build numba function to do this for efficiency with a for loop
            #

    #         print('recurns')
    #         print(recurns)

            rand4puttymod1 = np.random.randint(2, size=runs)
            rand4puttymod2 = np.random.randint(2, size=runs)



            recurns = rec_balls(recurns, picksEXS, rand4puttymod1, rand4puttymod2)

    #         print('recurns')
    #         print(recurns)


            picksEXS = np.reshape(picksEXS, [runs, exenumsig])
    #         print('picksEXS')
    #         print(picksEXS)





            # print('recurns')
            # print(recurns)

            # print('rand 1 & 2')
            # print(rand4puttymod1)
            # print(rand4puttymod2)


            recurns = np.reshape(recurns, [runs, 1])

            # print('recurns')
            # print(recurns)








            recurns = (np.concatenate((recit, recurns), axis=1)).T
            # print('recurns')
            # print(recurns)

            recweights_play = recweights[recurns[0], recurns[1]]
            # print('recweight_play')
            # print(recweights_play)




            # now use picksEXS to identify the appropriate exereceiver urn
            exerecurns = np.ravel_multi_index(picksEXS.T, indexedexerecurns.shape)
            exerecurns = np.vstack(exerecurns)
            exerecurns = (np.concatenate((exerecit, exerecurns), axis=1)).T
            # print('exerecurns')
            # print(exerecurns)

            exerecweights_play = exerecweights[exerecurns[0], exerecurns[1]]
            # print('exerecweight_play')
            # print(exerecweights_play)









            # to do weighted picks in parallel, we're going to normalize to values in [0, 1)
            # then generate a random number and see where it falls in the cumulative sum
            # of the normalized weights. Then we can just do a boolean comparison to get 
            # discrete values
            recweight_total = np.vstack(np.sum(recweights_play, axis = 1))
            recweight_norm = recweights_play / recweight_total
            recweight_norm = np.cumsum(recweight_norm, axis = 1)
            #     print(recweight_norm)
            rand4draws = np.random.rand(runs,1)
            #     print(rand4draws)
            draws = (rand4draws > recweight_norm).astype(int)

            draws = np.sum(draws, axis=1)
    #         print('draws')
    #         print(draws)

    #         print('BEFORE REINFORCEMENT')
    # #         print('sig_weights')
    # #         print(sig_weights)
    #         print('recweights')
    #         print(recweights)



            # to do weighted picksEXS in parallel, we're going to normalize to values in [0, 1)
            # then generate a random number and see where it falls in the cumulative sum
            # of the normalized weights. Then we can just do a boolean comparison to get 
            # discrete values
            exerecweight_total = np.vstack(np.sum(exerecweights_play, axis = 1))
            exerecweight_norm = exerecweights_play / exerecweight_total
            exerecweight_norm = np.cumsum(exerecweight_norm, axis = 1)
            #     print(exerecweight_norm)
            rand4exedraws = np.random.rand(runs,1)
            #     print(rand4exedraws)
            exedraws = (rand4exedraws > exerecweight_norm).astype(int)

            exedraws = np.sum(exedraws, axis=1)
    #         print('exedraws')
    #         print(exedraws)

    #         print('BEFORE REINFORCEMENT')
    #         print('exesig_weights')
    #         print(exesig_weights)
    #         print('exerecweights')
    #         print(exerecweights)




            ####################################################################################



            #final draws ##3**************************************************************************

            draws2check = putty_draws(draws, exedraws)
    #         print('draws2check')
    #         print('nature')
    #         print(nature)

    #         print('draws2check')
    #         print(draws2check)
            draws = np.reshape(draws, [runs, 1])
            exedraws = np.reshape(exedraws, [runs, 1])
            draws2check = np.reshape(draws2check, [runs, 1])
    #         print('draws')
    #         print(draws)




            nat4comp = np.reshape(nature, [runs, 1])
            #         print('nat4comp')
            #         print(nat4comp)

            success = (nat4comp == draws2check).astype(int)
    #         print('success')
    #         print(success)
    #         print('nature')
    #         print(nature)

    #         print('draws2check')
    #         print(draws2check)

            failures = (nat4comp != draws2check).astype(int)
            #         print('failures')
            #         print(failures)

            if it_01 > plays - 10000:
                cumsuc = cumsuc+success
            #         print('cumsuc')
            #         print(cumsuc)
            
            
            
            partialmod_suc = ((picksEXS+1)*sig_cost)*success

            success4r = success*rein
            success4r = success4r + partialmod_suc
            
            
            success4r = success4r.flatten()
            sigsuccess4r = success4r.copy()
            # print('success4r')
            # print(success4r)
            
            
            partialmod = picksEXS +1
#         print('partialmod')
#         print(partialmod)

            failures4p = (failures*partialmod)*sig_cost + failures*punish
#         print('failures4p')
#         print(failures4p)

            failures4p = failures4p.flatten()
            sigfailures4p = failures4p.copy()







            picks4learn = np.reshape(picks, [runs*numsig])
            # print('picks4learn')
            # print(picks4learn)
            picks4learn = np.vstack((pickslist, picks4learn))
            # print('picks4learn')
            # print(picks4learn)
            picks4learn_id = np.ravel_multi_index(picks4learn, (runs*numsig, numsigterms))
            # print('picks4learn_id')
            # print(picks4learn_id)

            picksEXS4learn = np.reshape(picksEXS, [runs*exenumsig])
            #         print('picksEXS4learn')
            #         print(picksEXS4learn)
            picksEXS4learn = np.vstack((picksEXSlist, picksEXS4learn))
            #         print('picksEXS4learn')
            #         print(picksEXS4learn)
            picksEXS4learn_id = np.ravel_multi_index(picksEXS4learn, (runs*exenumsig, exenumsigterms))


            #         print('sigweight_play')
            #         print(sigweight_play)

            #         print('recweights')
            #         print(recweights)



            # print('sigsuccess4r')
            # print(sigsuccess4r)
            # print('sigfailures4p')
            # print(sigfailures4p)
            # print('picksEXS and rand 1')
            # print(picksEXS)
            # print(rand4puttymod1)
            # print(len(picksEXS))


            exesigweight_play = exesigweight_play.flatten()
            exesigweight_play[picksEXS4learn_id] = exesigweight_play[picksEXS4learn_id] + success4r +failures4p

            exesigweight_play = np.reshape(exesigweight_play, [runs*exenumsig, exenumsigterms])




            exesig_weights[exesigweight_id] = exesigweight_play


    #         print('sigsuccess4r')
    #         print(sigsuccess4r)
    #         print('sigfailures4p')
    #         print(sigfailures4p)



            # function to correct for single sender
        
            sigsuccess4r = np.repeat(sigsuccess4r, numsig)
            sigfailures4p = np.repeat(sigfailures4p, numsig)

            

            sigsuccess4r, sigfailures4p = sig_cor(sigsuccess4r, sigfailures4p, picksEXS, rand4puttymod1)
            
            
    #         print('sigsuccess4r')
    #         print(sigsuccess4r)
    #         print('sigfailures4p')
    #         print(sigfailures4p)

            sigweight_play = sigweight_play.flatten()
            sigweight_play[picks4learn_id] = sigweight_play[picks4learn_id] + sigsuccess4r +sigfailures4p

            # print('sigweight play')
            # print(sigweight_play)





            sigweight_play = np.reshape(sigweight_play, [runs*numsig, numsigterms])




            sig_weights[sigweight_id] = sigweight_play

            #now do the same for the receiver

            draws4learn = np.reshape(draws, [runs])
    #         print('draws4learn')
    #         print(draws4learn)
            draws4learn = np.vstack((drawslist, draws4learn))

            draws4learn_id = np.ravel_multi_index(draws4learn, (runs, numrecterms))

            recweights_play = recweights_play.flatten()

            recweights_play[draws4learn_id] = recweights_play[draws4learn_id] +success4r +failures4p

            recweights_play = np.reshape(recweights_play, [runs, numrecterms])

            recweights[recurns[0], recurns[1]] = recweights_play


            # now make sure weights are at least 1
            sig_weights[sig_weights < 1.] = 1.
            recweights[recweights < 1.] = 1.
            # make sure weights are not out of bounds
#             sig_weights[sig_weights > bound] = bound
#             recweights[recweights > bound] = bound

            #now do the same for the exereceiver

            exedraws4learn = np.reshape(exedraws, [runs])
            exedraws4learn = np.vstack((exedrawslist, exedraws4learn))

            exedraws4learn_id = np.ravel_multi_index(exedraws4learn, (runs, exenumrecterms))

            exerecweights_play = exerecweights_play.flatten()

            exerecweights_play[exedraws4learn_id] = exerecweights_play[exedraws4learn_id] +success4r +failures4p

            exerecweights_play = np.reshape(exerecweights_play, [runs, exenumrecterms])

            exerecweights[exerecurns[0], exerecurns[1]] = exerecweights_play


            # now make sure weights are at least 1
            exesig_weights[exesig_weights < 1.] = 1.
            exerecweights[exerecweights < 1.] = 1.
            # make sure weights are not out of bounds
#             exesig_weights[exesig_weights > bound] = bound
#             exerecweights[exerecweights > bound] = bound




    #         print('AFTER REINFORCEMENT')
    # #         print('sig_weights')
    # #         print(sig_weights)
    #         print('recweights')
    #         print(recweights)
    #         print('exesig_weights')
    #         print(exesig_weights)
    #         print('exerecweights')
    #         print(exerecweights)
    
    
    
    
    
    # *************** Nash count mod ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    # Lets to a basic conversion of the weights into strategy profiles for
    # the nash counter to take as input

    xsend_profiles = np.zeros((runs, 6), dtype=int)
    asend_profiles = np.zeros((runs, 6), dtype=int)
    bsend_profiles = np.zeros((runs, 6), dtype=int)
    crec_profiles = np.zeros((runs, 4), dtype=int)
    xrec_profiles = np.zeros((runs, 2), dtype=int)

    xsend_max = np.zeros_like(exesig_weights)
    xsend_max[np.arange(len(exesig_weights)), exesig_weights.argmax(1)] = 1
    

    absend_max = np.zeros_like(sig_weights)
    absend_max[np.arange(len(sig_weights)), sig_weights.argmax(1)] = 1


    crec_max = np.zeros_like(recweights)
    crec_max[np.arange(runs)[:, None], np.arange(4)[None, :], np.argmax(recweights, axis=2)] = 1

    xrec_max = np.zeros_like(exerecweights)
    xrec_max[np.arange(runs)[:, None], np.arange(2)[None, :], np.argmax(exerecweights, axis=2)] = 1

    # print('xsend_max')
    # print(xsend_max)
    # print('absend_max')
    # print(absend_max)
    # print('crec_max')
    # print(crec_max)
    # print('xrec_max')
    # print(xrec_max)
    
    # lets get some stuff to check for the fake nash case
    
    abones_check = sig_weights.copy()
    xones_check = exesig_weights.copy()
    
    abones_check = np.sum(abones_check, axis=1)
    xones_check = np.sum(xones_check, axis=1)
    
    abones_check[abones_check > 2] = 0
    xones_check[xones_check > 2] = 0
    
    notnashones = 0
    notnashones_error = 0
    
    

    for it_03 in range(0, runs):
        ones_count = 0
        
        for it_04 in range(0, 6):
            xsend_profiles[it_03, it_04] = np.argmax(xsend_max[it_03*6+it_04])
            asend_profiles[it_03, it_04] = np.argmax(absend_max[it_03*12+it_04])
            bsend_profiles[it_03, it_04] = np.argmax(absend_max[it_03*12+6+it_04])
            ones_count = ones_count + xones_check[it_03*6+it_04] + abones_check[it_03*12+it_04] + abones_check[it_03*12+6+it_04]

        for it_05 in range(0, 4):
            crec_profiles[it_03, it_05] = np.argmax(crec_max[it_03, it_05])

        for it_06 in range(0, 2):
            xrec_profiles[it_03, it_06] = np.argmax(xrec_max[it_03, it_06])
            
        if ones_count > 0:
            notnashones = notnashones + 1
            if ones_count % 2 != 0:
                notnashones_error = notnashones_error +1


    # print('xsend_profiles')
    # print(xsend_profiles)
    # print('asend_profiles')
    # print(asend_profiles)
    # print('bsend_profiles')
    # print(bsend_profiles)
    # print('crec_profiles')
    # print(crec_profiles)
    # print('xrec_profiles')
    # print(xrec_profiles)


    # lets generate all the profiles

    siglist0 = np.zeros(6)
    siglist1 = np.ones(6)
    siglist = (np.concatenate((siglist0, siglist1))).astype(int)
    # print(siglist)

    sig_perms = list(itertools.permutations(siglist, 6))

    for it00 in range(len(sig_perms)):
        sig_perms[it00] = list(sig_perms[it00])

    sig_perms = np.array(sig_perms)
    sig_perms = np.unique(sig_perms, axis=0)
    # print(sig_perms)


    sigpermsexe = sig_perms
    sigpermsA = sig_perms
    sigpermsB = sig_perms

    reclist0 = np.zeros(4)
    reclist1 = np.ones(4)
    reclist2 = np.ones(4)*2
    reclist3 = np.ones(4)*3

    reclist = (np.concatenate((reclist0, reclist1, reclist2, reclist3))).astype(int)
    # print(reclist)

    recperms = list(itertools.permutations(reclist, 4))

    for it01 in range(len(recperms)):
        recperms[it01] = list(recperms[it01])

    recperms = np.array(recperms)
    recperms = np.unique(recperms, axis=0)
    # print(recperms)


    exereclist0 = np.zeros(2)
    exereclist1 = np.ones(2)

    exereclist = (np.concatenate((exereclist0, exereclist1))).astype(int)
    # print(exereclist)

    exerecperms = list(itertools.permutations(exereclist, 2))

    for it02 in range(len(exerecperms)):
        exerecperms[it02] = list(exerecperms[it02])

    exerecperms = np.array(exerecperms)
    exerecperms = np.unique(exerecperms, axis=0)
    # print(exerecperms)





    # now we need a function to check which profiles are Nash

    putty_nash, putty_notnash, nashtot = runs_nash_counter(xsend_profiles, asend_profiles, bsend_profiles, crec_profiles, xrec_profiles, runs, sigpermsexe, sigpermsA, sigpermsB, recperms, exerecperms)




    # END NASH COUNT MOD $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$









    # 
    # cumulative success rate
    # print(cumsuc)
    cumsuc = cumsuc/10000
    cumsucA[cumsuc < cutoffA] = 0
    cumsucA[cumsuc >= cutoffA] = 1
    cumsuccess_rateA = np.sum(cumsucA)/runs
    cumsucB[cumsuc < cutoffB] = 0
    cumsucB[cumsuc >= cutoffB] = 1
    cumsuccess_rateB = np.sum(cumsucB)/runs - cumsuccess_rateA
    cumsucC[cumsuc < cutoffC] = 0
    cumsucC[cumsuc >= cutoffC] = 1
    cumsuccess_rateC = np.sum(cumsucC)/runs - cumsuccess_rateA - cumsuccess_rateB



    
    return cumsuccess_rateA, cumsuccess_rateB, cumsuccess_rateC, putty_nash, putty_notnash, nashtot, notnashones, notnashones_error





In [4]:

start = time.perf_counter()


#nprocs = mp.cpu_count()-1
pool = mp.Pool(processes=5)

tile_count = 5


resAlist = []
resBlist = []
resClist = []
nash_list = []
notnash_list = []
nashtot_list = []
notnashones_list = []
notnashoneserror_list = []

for resA, resB, resC, resnash, resnotnash, restot, resones, resoneserror in pool.starmap(putty_tile, [(n, ) for n in range(tile_count)]):
    resAlist.append(resA)
    resBlist.append(resB)
    resClist.append(resC)
    nash_list.append(resnash)
    notnash_list.append(resnotnash)
    nashtot_list.append(restot)
    notnashones_list.append(resones)
    notnashoneserror_list.append(resoneserror)
    
    
    
resAlist = np.asarray(resAlist)
# print(real_results)
resAlist_mean = np.mean(resAlist)

resBlist = np.asarray(resBlist)
# print(real_results)
resBlist_mean = np.mean(resBlist)

resClist = np.asarray(resClist)
# print(real_results)
resClist_mean = np.mean(resClist)

nash_runs = np.sum(np.asarray(nash_list))
notnash_runs = np.sum(np.asarray(notnash_list))
nashtotal = np.sum(np.asarray(nashtot_list))

notnashones = np.sum(np.asarray(notnashones_list))
notnashoneserror = np.sum(np.asarray(notnashoneserror_list))


runs = 200

print(f'runs = {tile_count*runs}')

print(f'cumulative success rate over A = {resAlist_mean}')
print(f'cumulative success rate over B = {resBlist_mean}')
print(f'cumulative success rate over C = {resClist_mean}')
print(f'number of runs that were nash = {nash_runs}')
print(f'number of runs that were not nash = {notnash_runs}')
print(f'total number of runs = {nashtotal}')
print(f'potential overcount of nash = {notnashones}')
print(f'error parameter should be zero = {notnashoneserror}')






finish = time.perf_counter()
print(f'Finished in {round(finish-start,0)/60} minutes')

runs = 1000
cumulative success rate over A = 0.32199999999999995
cumulative success rate over B = 0.21300000000000002
cumulative success rate over C = 0.39999999999999997
number of runs that were nash = 948
number of runs that were not nash = 52
total number of runs = 1000
potential overcount of nash = 486
error parameter should be zero = 0
Finished in 324.81666666666666 minutes
